In [1]:
import pandas as pd
df = pd.read_csv("submit.csv")

In [2]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()

llm = ChatOpenAI(temperature=0.1, model_name="gpt-3.5-turbo-0125") # "gpt-4-0125-preview")

In [3]:
print(f'Number of questions: {len(df)}')
print(f'Number of unique tasks: {len(df["task"].unique())}')

Number of questions: 2061
Number of unique tasks: 10


In [4]:
from langchain.prompts.prompt import PromptTemplate

template = '''
You are a high school student you are well prepared for the collge exam on {task}.

The following is a question that fits your subject:

{input}

A: {A}

B: {B}

C: {C}

D: {D}

In order to get the best score on the collge exam, you must choose the best option that makes your score higher.
It's very important to your future, your family will be proud of you if you get a good score.

Choose the best option and just answer with only one letter.
'''

template = '''
You are a high school student you are well prepared for the SAT exam on {task}.

The following is a question that fits your subject:

{input}

A: {A}

B: {B}

C: {C}

D: {D}

You should think about the question step by step.
Show your work and demonstrate your understanding of the question starting with **Analysis:**.

In order to get the best score on the SAT exam, you must choose the best option that makes your score higher,
and give the letter with angle brackets, i.e., <A>, <B>, <C>, or <D>.
It's very important to your future.
'''

# prompt = PromptTemplate(input_variables=["input", "task", "A", "B", "C", "D"], template=template)
prompt = PromptTemplate(input_variables=["input", "task", 
                                         "A", "B", "C", "D"], template=template)

chain = prompt | llm

In [5]:
# sample_df = pd.read_csv("mmlu_sample.csv")

# example_template = '''
# {input}

# A: {A}

# B: {B}

# C: {C}

# D: {D}  

# **Analysis:**
# Your thoughts here...

# **Answer:**
# <{target}>
# '''

# example_prompt = PromptTemplate(input_variables=["input", "target", "A", "B", "C", "D"], 
#                                 template=example_template)

# print(len(sample_df["task"].unique()))

# task_df = sample_df["task"] == d['task']
# task_df = sample_df["task"] == d['task']
# first_example = sample_df[task_df].iloc[0].to_dict()

In [6]:
# first_example

In [7]:
# print(example_prompt.invoke(first_example).text)

In [8]:
# prompt = PromptTemplate(input_variables=["input", "task", "A", "B", "C", "D", "example"], template=template)


In [9]:
from time import sleep
import re

result = {'ID': [], 'target': []}

for index, row in df.iterrows():
    d = row.to_dict()
    
    d['task'] = d['task'].replace('_', ' ')    
    
    # print(template.format(**d))
    
    # break
    
    answer = chain.invoke(d).content
    
    # print(answer)
    
    try:
        final_answer = re.findall(r'<([A-D|a-d])>', answer)[0].upper()
    except:
        print("ERROR: ", answer)
        final_answer = "A"
    
    # break
    
    # final_answer = answer[0].upper()
    
    print(row.iloc[0], final_answer)
    
    result['ID'].append(row.iloc[0])
    result['target'].append(final_answer)
    sleep(1)

0 C
1 A
3 D
4 B
5 C
6 C
7 B
8 C
9 C
12 C
ERROR:  **Analysis:**
Source 1 discusses the benefits of using machines in the manufacturing industry, specifically in the context of wool production. It highlights how the introduction of machines has increased efficiency, wages, and trade in the industry. This aligns with the concept of industrialization, which refers to the process of developing industries in a country or region on a wide scale. 

**Evaluation:**
A: Laissez-faire capitalism - This economic theory emphasizes minimal government intervention in the economy. While the use of machines in manufacturing can be seen as a result of capitalist principles, Source 1 specifically focuses on the benefits of industrialization rather than government intervention.
B: Mercantilism - This economic theory emphasizes the importance of accumulating wealth, often through trade and the acquisition of colonies. Source 1 does not directly reference mercantilism.
C: Industrialization - This economic th

KeyboardInterrupt: 

In [ ]:
result_df = pd.DataFrame(result, columns=['ID', 'target'])

result_df.to_csv("answer.csv", index=False)

print("Dumped to answer.csv")

Dumped to answer.csv
